In [1]:
from sklearn.metrics import classification_report
from tensorflow.python.keras import optimizers
from tensorflow.python.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.utils.layer_utils import print_summary
from tensorflow.python.keras.models import Model as KerasModel
from tensorflow.python.keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Reshape, Concatenate, LeakyReLU
from tensorflow.python.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
import numpy as np
import os
import random
random.seed(32)
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0" # Choose GPU NUMBERS [0, 1, 2, 3]
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
K.set_session(sess)
sess.run(tf.global_variables_initializer())

In [2]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        # shear_range=0.2,
        # zoom_range=0.2,
        # horizontal_flip=True
)

validation_datagen = ImageDataGenerator(
        rescale=1./255,
)
train_generator = train_datagen.flow_from_directory(
        'datasets/Face2Face/train',# this is the target directory
        target_size=(256, 256),  # all images will be resized to 150x150
        class_mode='binary',
        shuffle=True,
        seed=32) 
validation_generator = validation_datagen.flow_from_directory(
        'datasets/Face2Face/val',# this is the target directory
        target_size=(256, 256),  # all images will be resized to 150x150
        class_mode='binary',
        shuffle=True,
        seed=32) 

In [3]:
x=train_generator.__getitem__(0)
print(len(train_generator.labels),train_generator.labels)
print(x[1].shape)
print(x[1])
plt.imshow(x[0][7])
plt.title(x[1][7])


In [4]:
def get_pred(predictions):
    pred=[]
    for p in predictions:
        if p<0.50:
             pred.append(0)
        else:
            pred.append(1)
    return pred

IMGWIDTH = 256

class Classifier:
    def __init__(self):
        self.model = 0
    
    def predict(self, x):
        return self.model.predict(x)
    
    def fit(self, x, y):
        return self.model.train_on_batch(x, y)
    
    def get_accuracy(self, x, y):
        return self.model.test_on_batch(x, y)
    
    def load(self, path):
        self.model.load_weights(path)


class MesoInception4(Classifier):
    def __init__(self, learning_rate = 0.001):
        self.model = self.init_model()
        optimizer = Adam(lr = learning_rate)
        self.model.compile(optimizer = optimizer, loss = 'mean_squared_error', metrics = ['accuracy'])
    
    def InceptionLayer(self, a, b, c, d):
        def func(x):
            x1 = Conv2D(a, (1, 1), padding='same', activation='relu')(x)
            
            x2 = Conv2D(b, (1, 1), padding='same', activation='relu')(x)
            x2 = Conv2D(b, (3, 3), padding='same', activation='relu')(x2)
            
            x3 = Conv2D(c, (1, 1), padding='same', activation='relu')(x)
            x3 = Conv2D(c, (3, 3), dilation_rate = 2, strides = 1, padding='same', activation='relu')(x3)
            
            x4 = Conv2D(d, (1, 1), padding='same', activation='relu')(x)
            x4 = Conv2D(d, (3, 3), dilation_rate = 3, strides = 1, padding='same', activation='relu')(x4)

            y = Concatenate(axis = -1)([x1, x2, x3, x4])
            
            return y
        return func
    
    def init_model(self):
        x = Input(shape = (IMGWIDTH, IMGWIDTH, 3))
        
        x1 = self.InceptionLayer(1, 4, 4, 2)(x)
        x1 = BatchNormalization()(x1)
        x1 = MaxPooling2D(pool_size=(2, 2), padding='same')(x1)
        
        x2 = self.InceptionLayer(2, 4, 4, 2)(x1)
        x2 = BatchNormalization()(x2)
        x2 = MaxPooling2D(pool_size=(2, 2), padding='same')(x2)        
        
        x3 = Conv2D(16, (5, 5), padding='same', activation = 'relu')(x2)
        x3 = BatchNormalization()(x3)
        x3 = MaxPooling2D(pool_size=(2, 2), padding='same')(x3)
        
        x4 = Conv2D(16, (5, 5), padding='same', activation = 'relu')(x3)
        x4 = BatchNormalization()(x4)
        x4 = MaxPooling2D(pool_size=(4, 4), padding='same')(x4)
        
        y = Flatten()(x4)
        y = Dropout(0.5)(y)
        y = Dense(16)(y)
        y = LeakyReLU(alpha=0.1)(y)
        y = Dropout(0.5)(y)
        y = Dense(1, activation = 'sigmoid')(y)

        return KerasModel(inputs = x, outputs = y)

In [5]:
model= MesoInception4().model
model_checkpoint_callback = ModelCheckpoint(
    filepath="MI4_F2F_Ebest.h5",
    save_weights_only=True,
    monitor='val_acc',
    mode='max',
    save_best_only=True)
csv_logger = CSVLogger("MI4_F2F_train.csv", append=True, separator=',')
print_summary(model, line_length=115, positions=None, print_fn=None)

In [6]:
model.fit_generator(train_generator,
                    verbose=2,
                    epochs=50,
                    validation_data=validation_generator,
                    callbacks=[model_checkpoint_callback,csv_logger])
model.save_weights("MI4_F2F_E50.h5")

# DeepFakeDetection Test 

In [7]:
test_datagen = ImageDataGenerator(rescale=1./255)
model= MesoInception4()
model.load("MI4_F2F_Ebest.h5")
model=model.model
print_summary(model, line_length=115, positions=None, print_fn=None)
test_generator = test_datagen.flow_from_directory(
        'datasets/DeepFakeDetection/test',# this is the target directory
        target_size=(256, 256),  # all images will be resized to 150x150
        class_mode='binary',
        shuffle=False,
        seed=32) 
predictions=model.evaluate_generator(test_generator,verbose=2)
print(predictions)
predictions=model.predict_generator(test_generator,verbose=2)
predicted_classes = get_pred(predictions)
true_classes = test_generator.classes
report = classification_report(true_classes, predicted_classes, digits=4)
print(report)

# DeepFake Test 

In [8]:
test_datagen = ImageDataGenerator(rescale=1./255)
model= MesoInception4()
model.load("MI4_F2F_Ebest.h5")
model=model.model
print_summary(model, line_length=115, positions=None, print_fn=None)
test_generator = test_datagen.flow_from_directory(
        'datasets/DeepFake/test',# this is the target directory
        target_size=(256, 256),  # all images will be resized to 150x150
        class_mode='binary',
        shuffle=False,
        seed=32) 
predictions=model.evaluate_generator(test_generator,verbose=2)
print(predictions)
predictions=model.predict_generator(test_generator,verbose=2)
predicted_classes = get_pred(predictions)
true_classes = test_generator.classes
report = classification_report(true_classes, predicted_classes, digits=4)
print(report)

# FaceSwap Test 

In [9]:
test_datagen = ImageDataGenerator(rescale=1./255)
model= MesoInception4()
model.load("MI4_F2F_Ebest.h5")
model=model.model
print_summary(model, line_length=115, positions=None, print_fn=None)
test_generator = test_datagen.flow_from_directory(
        'datasets/FaceSwap/test',# this is the target directory
        target_size=(256, 256),  # all images will be resized to 150x150
        class_mode='binary',
        shuffle=False,
        seed=32) 
predictions=model.evaluate_generator(test_generator,verbose=2)
print(predictions)
predictions=model.predict_generator(test_generator,verbose=2)
predicted_classes = get_pred(predictions)
true_classes = test_generator.classes
report = classification_report(true_classes, predicted_classes, digits=4)
print(report)

# Face2Face Test 

In [10]:
test_datagen = ImageDataGenerator(rescale=1./255)
model= MesoInception4()
model.load("MI4_F2F_Ebest.h5")
model=model.model
print_summary(model, line_length=115, positions=None, print_fn=None)
test_generator = test_datagen.flow_from_directory(
        'datasets/Face2Face/test',# this is the target directory
        target_size=(256, 256),  # all images will be resized to 150x150
        class_mode='binary',
        shuffle=False,
        seed=32) 
predictions=model.evaluate_generator(test_generator,verbose=2)
print(predictions)
predictions=model.predict_generator(test_generator,verbose=2)
predicted_classes = get_pred(predictions)
true_classes = test_generator.classes
report = classification_report(true_classes, predicted_classes, digits=4)
print(report)

# NeuralTextures Test 

In [11]:
test_datagen = ImageDataGenerator(rescale=1./255)
model= MesoInception4()
model.load("MI4_F2F_Ebest.h5")
model=model.model
print_summary(model, line_length=115, positions=None, print_fn=None)
test_generator = test_datagen.flow_from_directory(
        'datasets/NeuralTextures/test',# this is the target directory
        target_size=(256, 256),  # all images will be resized to 150x150
        class_mode='binary',
        shuffle=False,
        seed=32) 
predictions=model.evaluate_generator(test_generator,verbose=2)
print(predictions)
predictions=model.predict_generator(test_generator,verbose=2)
predicted_classes = get_pred(predictions)
true_classes = test_generator.classes
report = classification_report(true_classes, predicted_classes, digits=4)
print(report)

# DeepFakewild Test 

In [12]:
test_datagen = ImageDataGenerator(rescale=1./255)
model= MesoInception4()
model.load("MI4_F2F_Ebest.h5")
model=model.model
print_summary(model, line_length=115, positions=None, print_fn=None)
test_generator = test_datagen.flow_from_directory(
        'datasets/DeepFakewild/test',# this is the target directory
        target_size=(256, 256),  # all images will be resized to 150x150
        class_mode='binary',
        shuffle=False,
        seed=32) 
predictions=model.evaluate_generator(test_generator,verbose=2)
print(predictions)
predictions=model.predict_generator(test_generator,verbose=2)
predicted_classes = get_pred(predictions)
true_classes = test_generator.classes
report = classification_report(true_classes, predicted_classes, digits=4)
print(report)

# DFDC Test 

In [13]:
test_datagen = ImageDataGenerator(rescale=1./255)
model= MesoInception4()
model.load("MI4_F2F_Ebest.h5")
model=model.model
print_summary(model, line_length=115, positions=None, print_fn=None)
test_generator = test_datagen.flow_from_directory(
        'datasets/DFDC/test',# this is the target directory
        target_size=(256, 256),  # all images will be resized to 150x150
        class_mode='binary',
        shuffle=False,
        seed=32) 
predictions=model.evaluate_generator(test_generator,verbose=2)
print(predictions)
predictions=model.predict_generator(test_generator,verbose=2)
predicted_classes = get_pred(predictions)
true_classes = test_generator.classes
report = classification_report(true_classes, predicted_classes, digits=4)
print(report)